In [51]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time




In [11]:
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")
driver = webdriver.Chrome(options=chrome_options)


In [52]:
#open page
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://digital.oegbverlag.at/login?r=https%3A%2F%2Fdigital.oegbverlag.at%2Fkvsystem")

#for some reason I have to manually click on the "reject cookies" option, can't find a way to make the webdriver do it 

In [45]:
#select "für ögbverlag kunden"
element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div/div[1]/div[3]/div[2]/button")))
select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div[1]/div[3]/div[2]/button')
select_element.click()

In [46]:
#login
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "password")

username.send_keys("jakob.wall@econ.uzh.ch")
password.send_keys("D00rcityoverhere")

driver.find_element(By.ID, "login").click()

In [47]:
#select "erweiterte Suche and search"
element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]")))
select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]')
select_element.click()


In [48]:
searchbar = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[1]/div/input")
searchbar.send_keys(Keys.RETURN)


In [50]:
#click on "mehr anzeigen" until we have the full list
#function that checks if "mehr anzeigen" exists
def check_element_exists_xpath(xpath):
    try:
        # Try to find the element
        driver.find_element(By.XPATH, xpath)
        return True
    except NoSuchElementException:
        return False

#select "mehr anzeigen"
while True:
    if check_element_exists_xpath('/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button'):
        element=WebDriverWait(driver, 30000000).until(EC. element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button")))
        select_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button')
        select_element.click()
        time.sleep(5)
    else:
        break

    

KeyboardInterrupt: 

In [42]:
def check_element_exists_xpath(xpath):
    try:
        # Try to find the element
        driver.find_element(By.XPATH, xpath)
        return True
    except NoSuchElementException:
        return False
    

check_element_exists_xpath('/html/body/div[1]/div/div[2]/div[2]/div/div[2]/button')

True